## Setup 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tqdm import tqdm

print("\u2022 Using TensorFlow Version:", tf.__version__)
print("\u2022 Using TensorFlow Hub Version: ", hub.__version__)
print('\u2022 GPU Device Found.' if tf.test.is_gpu_available() else '\u2022 GPU Device Not Found. Running on CPU')

• Using TensorFlow Version: 2.8.2
• Using TensorFlow Hub Version:  0.12.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
• GPU Device Found.


In [ ]:
# Define the training and validation base directories
train_dir = '/content/drive/MyDrive/Accident_Classification/data_photo/Training'

# Directory with training horse pictures
train_accident_dir = os.path.join(train_dir, 'Accident')
# Directory with training humans pictures
train_no_accident_dir = os.path.join(train_dir, 'No_Accident')


# Check the number of images for each class and set
print(f"There are {len(os.listdir(train_accident_dir))} images of accident for training.\n")
print(f"There are {len(os.listdir(train_no_accident_dir))} images of no accident for training.\n")

There are 110 images of accident for training.

There are 110 images of no accident for training.



In [ ]:
validation_dir = '/content/drive/MyDrive/Accident_Classification/data_photo/Testing'

# Directory with validation horse pictures
validation_accident_dir = os.path.join(validation_dir, 'Accident')
# Directory with validation human pictures
validation_no_accident_dir = os.path.join(validation_dir, 'No_Accident')

print(f"There are {len(os.listdir(validation_accident_dir))} images of accident for validation.\n")
print(f"There are {len(os.listdir(validation_no_accident_dir))} images of no accident for validation.\n")

There are 40 images of accident for validation.

There are 40 images of no accident for validation.



In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32, 
                                                      class_mode='binary',
                                                      target_size=(350, 350))

  validation_datagen = ImageDataGenerator(rescale=1/255)

  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32, 
                                                                class_mode='binary',
                                                                target_size=(350, 350))
  return train_generator, validation_generator

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [ ]:
train_generator, validation_generator = train_val_generators(train_dir, validation_dir)

Found 220 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [ ]:
from tensorflow import keras

In [ ]:
base_model = keras.applications.MobileNetV2(
    weights='imagenet', 
    input_shape=(350, 350, 3),
    include_top=False)

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 350, 350, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 175, 175, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 175, 175, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

In [ ]:
inputs = keras.Input(shape=(350, 350, 3))
x = base_model(inputs, training=False)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(2, activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer = 'adam', 
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
callbacks = myCallback()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs = 100,
                    verbose = 1,
                    callbacks=callbacks)

Epoch 1/100
7/7 [==============================] - 12s 1s/step - loss: 18.8824 - accuracy: 0.5273 - val_loss: 11.0757 - val_accuracy: 0.5250
Epoch 2/100
7/7 [==============================] - 9s 1s/step - loss: 9.1418 - accuracy: 0.6227 - val_loss: 0.7813 - val_accuracy: 0.9250
Epoch 3/100
7/7 [==============================] - 9s 1s/step - loss: 3.7463 - accuracy: 0.7727 - val_loss: 1.1067 - val_accuracy: 0.8500
Epoch 4/100
7/7 [==============================] - 9s 1s/step - loss: 1.8161 - accuracy: 0.8591 - val_loss: 0.6325 - val_accuracy: 0.9125
Epoch 5/100
7/7 [==============================] - 9s 1s/step - loss: 0.5798 - accuracy: 0.8955 - val_loss: 0.2544 - val_accuracy: 0.9250
Epoch 6/100
7/7 [==============================] - 9s 1s/step - loss: 0.2992 - accuracy: 0.8682 - val_loss: 0.1840 - val_accuracy: 0.9500
Epoch 7/100
7/7 [==============================] - 9s 1s/step - loss: 0.1710 - accuracy: 0.9500 - val_loss: 0.1771 - val_accuracy: 0.9375
Epoch 8/100
7/7 [==============

## Export the Model

In [ ]:
!mkdir -p exp_saved_model
model.save('exp_saved_model/model')

INFO:tensorflow:Assets written to: exp_saved_model/model/assets


INFO:tensorflow:Assets written to: exp_saved_model/model/assets


In [ ]:
saved_model = tf.keras.models.load_model('/content/exp_saved_model/model')

# Check its architecture
saved_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 350, 350, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 11, 11, 1280)     2257984   
 ional)                                                          
                                                                 
 flatten_1 (Flatten)         (None, 154880)            0         
                                                                 
 dense_2 (Dense)             (None, 128)               19824768  
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                           

In [ ]:
%%bash -s $SAVED_MODEL
saved_model_cli show --dir $1 --tag_set serve --signature_def serving_default

Traceback (most recent call last):
  File "/usr/local/bin/saved_model_cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 1260, in main
    args.func(args)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 745, in show
    _show_inputs_outputs(args.dir, args.tag_set, args.signature_def)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_cli.py", line 152, in _show_inputs_outputs
    tag_set)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_utils.py", line 113, in get_meta_graph_def
    saved_model = read_saved_model(saved_model_dir)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/tools/saved_model_utils.py", line 51, in read_saved_model
    raise IOError("SavedModel file does not exist at: %s" % saved_model_dir)
OSError: SavedModel file does not exist at: $SAVED_MODEL

In [ ]:
loaded = tf.saved_model.load('/content/exp_saved_model/model')

In [ ]:
print(list(loaded.signatures.keys()))
infer = loaded.signatures["serving_default"]
print(infer.structured_input_signature)
print(infer.structured_outputs)

['serving_default']
((), {'input_3': TensorSpec(shape=(None, 350, 350, 3), dtype=tf.float32, name='input_3')})
{'dense_3': TensorSpec(shape=(None, 2), dtype=tf.float32, name='dense_3')}


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/exp_saved_model/model',signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()
tflite_model_file = 'converted_model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)
